# LangSmith and Evaluation Overview with AI Makerspace

Today we'll be looking at an amazing tool:

[LangSmith](https://docs.smith.langchain.com/)!

This tool will help us monitor, test, debug, and evaluate our LangChain applications - and more!

We'll also be looking at a few Advanced Retrieval techniques along the way - and evaluate it using LangSmith!

✋BREAKOUT ROOM #1:
- Task 1: Dependencies and OpenAI API Key
- Task 2: Basic RAG Chain
- Task 3: Setting Up LangSmith
- Task 4: Examining the Trace in LangSmith!
- Task 5: Create Testing Dataset

✋BREAKOUT ROOM #2:
- Task 1: Parent Document Retriever
- Task 2: Ensemble Retriever

## Task 1: Dependencies and OpenAI API Key

We'll be using OpenAI's suite of models today to help us generate and embed our documents for a simple RAG system built on top of LangChain's blogs!

In [63]:
!pip install langchain_core langchain_openai langchain_community langchain-qdrant qdrant-client langsmith openai tiktoken cohere -qU

In [64]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

## Task 2: Basic RAG Chain

Now we'll set up our basic RAG chain, first up we need a model!

### OpenAI Model


We'll use OpenAI's `gpt-3.5-turbo` model to ensure we can use a stronger model for decent evaluation later!

Notice that we can tag our resources - this will help us be able to keep track of which resources were used where later on!

In [65]:
from langchain_openai.chat_models import ChatOpenAI

base_llm = ChatOpenAI(model="gpt-3.5-turbo", tags=["base_llm"])

#### Asyncio Bug Handling

This is necessary for Colab.

In [66]:
import nest_asyncio
nest_asyncio.apply()

### SiteMap Loader

We'll use a SiteMapLoader to scrape the LangChain blogs.

In [67]:
!pip install lxml

from langchain.document_loaders import SitemapLoader

documents = SitemapLoader(web_path="https://blog.langchain.dev/sitemap-posts.xml").load()

Fetching pages: 100%|##########| 219/219 [00:04<00:00, 52.52it/s]


In [68]:
documents[0].metadata["source"]

'https://blog.langchain.dev/langgraph-cloud/'

### RecursiveCharacterTextSplitter

We're going to use a relatively naive text splitting strategy today!

In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

split_documents = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 500,
    chunk_overlap = 20
).split_documents(documents)

In [70]:
len(split_documents)

1536

### Embeddings

We'll be leveraging OpenAI's [text-embedding-3-small](https://openai.com/index/new-embedding-models-and-api-updates/) today!

In [71]:
from langchain_openai.embeddings import OpenAIEmbeddings

base_embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

### Qdrant VectorStore Retriever

Now we can use a Qdrant VectorStore to embed and store our documents and then convert it to a retriever so it can be used in our chain!

In [72]:
from langchain_qdrant import Qdrant

vectorstore = Qdrant.from_documents(
    split_documents,
    base_embeddings_model,
    location=":memory:",
    collection_name="langchainblogs")

In [73]:
base_retriever = vectorstore.as_retriever()

### Prompt Template

All we have left is a prompt template, which we'll create here!

In [74]:
from langchain.prompts import ChatPromptTemplate

base_rag_prompt_template = """\
Using the provided context, please answer the user's question. If you don't know the answer based on the context, say you don't know.

Context:
{context}

Question:
{question}
"""

base_rag_prompt = ChatPromptTemplate.from_template(base_rag_prompt_template)

### LCEL Chain

Now that we have:

- Embeddings Model
- Generation Model
- Retriever
- Prompt

We're ready to build our LCEL chain!

Keep in mind that we're returning our source documents with our queries - while this isn't necessary, it's a great thing to get into the habit of doing.

In [75]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

base_rag_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": base_rag_prompt | base_llm | StrOutputParser(), "context": itemgetter("context")}
)

Let's test it out!

In [76]:
base_rag_chain.invoke({"question" : "What is a good way to evaluate agents?"})["response"]

"A good way to evaluate agents is by testing their capabilities in tasks such as planning, task decomposition, function calling, and the ability to override pre-trained biases when needed. Additionally, assessing an agent's performance across different tasks and considering its ability to generalize reliably for various workflows where reasoning is required can be helpful in evaluating agents effectively."

## Task 3: Setting Up LangSmith

Now that we have a chain - we're ready to get started with LangSmith!

We're going to go ahead and use the following `env` variables to get our Colab notebook set up to start reporting.

If all you needed was simple monitoring - this is all you would need to do!

In [77]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangSmith - {unique_id}"

### LangSmith API

In order to use LangSmith - you will need a beta key, you can join the queue through the `Beta Sign Up` button on LangSmith's homepage!

Join [here](https://www.langchain.com/langsmith)

In [78]:
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

Let's test our our first generation!

In [79]:
base_rag_chain.invoke({"question" : "What is LangSmith?"}, {"tags" : ["Demo Run"]})['response']

'LangSmith is a unified platform for debugging, testing, evaluating, and monitoring LLM applications. It allows users to easily debug, monitor, test, evaluate, and share their LLM applications.'

## Task 4: Examining the Trace in LangSmith!

Head on over to your LangSmith web UI to check out how the trace looks in LangSmith!

#### 🏗️ Activity #1:

Include a screenshot of your trace and explain what it means.

![image](https://i.imgur.com/WQtlxc5.png)

## Task 5: Create Testing Dataset

Now we can create a dataset using some user defined questions, and providing the retrieved context as a "ground truth" context.

> NOTE: There are many different ways you can approach this specific task - generating ground truth answers with AI, using human experts to generate golden datasets, and more!

### Synthetic Data Generation (SDG)

In order to full test our RAG chain, and the various modifications we'll be using in the following notebook, we'll need to create a small synthetic dataset that is relevant to our task!

Let's start by generating a series of questions - which begins with a simple model definition!

> NOTE: We're going to be using a purposefully simplified datagen pipeline as an example today - but you could leverage the RAGAS SDG pipeline just as easily.

In [80]:
question_model = ChatOpenAI(model="gpt-3.5-turbo")

Next up, we'll create some novel chunks from our source data!

In [81]:
sdg_documents = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000,
    chunk_overlap = 20
).split_documents(documents)

Now, let's ask some questions that could be answered from the provided chunks!

In [82]:
question_prompt_template = """\
You are a University Professor creating questions for an exam. You must create a question for a given piece of context.

The question must be answerable only using the provided context.

Avoid creating questions that are ambiguous or vague. They should be specifically related to the context.

Your output must only be the question.

Context:
{context}
"""

question_prompt = ChatPromptTemplate.from_template(question_prompt_template)

In [83]:
question_chain = question_prompt | question_model

Now we can loop through a subset of our context chunks and create question/context pairs!

In [84]:
from tqdm import tqdm
import json

question_context_pairs = []

for idx in tqdm(range(0, len(sdg_documents), 40)):
  question = question_chain.invoke({"context" : sdg_documents[idx].page_content})
  question_context_pairs.append({"question" : question.content, "context" : sdg_documents[idx].page_content, "idx" : idx})


100%|██████████| 22/22 [00:15<00:00,  1.45it/s]


In [85]:
question_context_pairs[0]

{'question': 'What is the name of the new infrastructure for running agents at scale that is available in beta, mentioned in the provided context?',
 'context': 'Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale, reliably\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAnnouncing LangGraph v0.1 & LangGraph Cloud: Running agents at scale, reliably\nOur new infrastructure for running agents at scale, LangGraph Cloud, is available in beta. We also have a new stable release of LangGraph.\n\n6 min read\nJun 27, 2024',
 'idx': 0}

We'll repeat this process for answers!

In [86]:
answer_prompt_template = """\
You are a University Professor creating an exam. You must create a answer for a given piece of context and question.

The answer must only rely on the provided context.

Your output must only be the answer.

Context:
{context}
"""

answer_prompt = ChatPromptTemplate.from_template(answer_prompt_template)

In [87]:
answer_chain = answer_prompt | question_model

In [88]:
for question_context_pair in tqdm(question_context_pairs):
  question_context_pair["answer"] = answer_chain.invoke({"question" : question_context_pair["question"], "context" : question_context_pair["context"]}).content

100%|██████████| 22/22 [00:14<00:00,  1.47it/s]


In [89]:
question_context_pairs[0]

{'question': 'What is the name of the new infrastructure for running agents at scale that is available in beta, mentioned in the provided context?',
 'context': 'Announcing LangGraph v0.1 & LangGraph Cloud: Running agents at scale, reliably\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAnnouncing LangGraph v0.1 & LangGraph Cloud: Running agents at scale, reliably\nOur new infrastructure for running agents at scale, LangGraph Cloud, is available in beta. We also have a new stable release of LangGraph.\n\n6 min read\nJun 27, 2024',
 'idx': 0,
 'answer': 'LangGraph v0.1 & LangGraph Cloud.'}

Now we can set up our LangSmith client - and we'll add the above created dataset to our LangSmith instance!

> NOTE: Read more about this process [here](https://docs.smith.langchain.com/old/evaluation/faq/manage-datasets#create-from-list-of-values)

In [90]:
from langsmith import Client

client = Client()

dataset_name = "langsmith-demo-dataset-v2"

dataset = client.create_dataset(
    dataset_name=dataset_name, description="LangChain Blog Test Questions"
)

for triplet in question_context_pairs:
  client.create_example(
      inputs={"question" : triplet["question"]},
      outputs={"answer" : triplet["answer"]},
      dataset_id=dataset.id
  )

### Evaluation

Now we can run the evaluation!

We'll need to start by preparing some custom data preparation functions to ensure our chain works with the expected inputs/outputs from the `evaluate` process in LangSmith.

> NOTE: More reading on this available [here](https://docs.smith.langchain.com/how_to_guides/evaluation/evaluate_llm_application#evaluate-a-langchain-runnable)

In [91]:
def prepare_data_ref(run, example):
  return {
      "prediction" : run.outputs["response"],
      "reference" : example.outputs["answer"],
      "input" : example.inputs["question"]
  }

def prepare_data_noref(run, example):
  return {
      "prediction" : run.outputs["response"],
      "input" : example.inputs["question"]
  }

We'll be using a few custom evaluators to evaluate our pipeline, as well as a few "built in" methods!

Check out the built-ins [here](https://docs.smith.langchain.com/reference/sdk_reference/langchain_evaluators)!

In [34]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

cot_qa_evaluator = LangChainStringEvaluator("cot_qa", prepare_data=prepare_data_ref)

unlabeled_dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria" : {
            "dopeness" : "Is the answer to the question dope, meaning cool - awesome - and legit?"
        }
    },
    prepare_data=prepare_data_noref
)

labeled_score_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": "Is the generated answer the same as the reference answer?"
        },
    },
    prepare_data=prepare_data_ref
)

unlabeled_coherence_evaluator = LangChainStringEvaluator("criteria", config={"criteria": "coherence"}, prepare_data=prepare_data_noref)
labeled_relevance_evaluator = LangChainStringEvaluator("labeled_criteria", config={ "criteria": "relevance"}, prepare_data=prepare_data_ref)

base_rag_results = evaluate(
    base_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        cot_qa_evaluator,
        unlabeled_dopeness_evaluator,
        labeled_score_evaluator,
        unlabeled_coherence_evaluator,
        labeled_relevance_evaluator
        ],
    experiment_prefix="Base RAG Evaluation"
)

View the evaluation results for experiment: 'Base RAG Evaluation-2507d718' at:
https://smith.langchain.com/o/3e2d3fa4-8d65-5328-ab1f-a089d874c8f0/datasets/5f75ca1f-b2ae-4821-9cf0-9ea7ca5ec2ae/compare?selectedSessions=11677221-de95-46df-a009-0e66a93f2cff




0it [00:00, ?it/s]

Error running evaluator <DynamicRunEvaluator evaluate> on run 1fec6ab4-82a6-4fab-9a14-29bb7f5d1aaf: RateLimitError("Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-AdhcHEVRXYxcJVZh94lEyz9i on tokens per min (TPM): Limit 10000, Used 9939, Requested 243. Please try again in 1.092s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}")
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/evaluation/_runner.py", line 1233, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/evaluation/evaluator.py", line 278, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/run_helpers.py", line 568, in wrapper
    raise e
  File "/opt/

## Testing Other Retrievers

Now we can test our how changing our Retriever impacts our LangSmith evaluation!

We'll build this simple qa_chain factory to create standardized qa_chains where the only different component will be the retriever.

In [92]:
def create_qa_chain(retriever):
  primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
  created_qa_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": base_rag_prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )
  return created_qa_chain

### Task 1: Parent Document Retriever

One of the easier ways we can imagine improving a retriever is to embed our documents into small chunks, and then retrieve a significant amount of additional context that "surrounds" the found context.

You can read more about this method [here](https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever)!

The basic outline of this retrieval method is as follows:

1. Obtain User Question
2. Retrieve child documents using Dense Vector Retrieval
3. Merge the child documents based on their parents. If they have the same parents - they become merged.
4. Replace the child documents with their respective parent documents from an in-memory-store.
5. Use the parent documents to augment generation.

In [93]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(":memory:")
client.create_collection(
    collection_name="split_parents",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vectorstore = Qdrant(client, collection_name="split_parents", embeddings=base_embeddings_model)

store = InMemoryStore()

In [94]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=RecursiveCharacterTextSplitter(chunk_size=400),
    parent_splitter=RecursiveCharacterTextSplitter(chunk_size=2000),
)

In [95]:
parent_document_retriever.add_documents(documents)

Let's create, test, and then evaluate our new chain!

In [96]:
parent_document_retriever_qa_chain = create_qa_chain(parent_document_retriever)

In [97]:
parent_document_retriever_qa_chain.invoke({"question" : "What is RAG?"})["response"].content

'RAG stands for Retrieval Augmented Generation, which is a central paradigm in Large Language Model (LLM) application development. It involves connecting LLMs to external data sources to retrieve relevant documents based on a user query and generate an answer grounded in the retrieved context.'

#### Evaluating the Parent Document Retrieval Pipeline

Now that we've created a new retriever - let's try evaluating it on the same dataset!

In [98]:
pdr_rag_results = evaluate(
    parent_document_retriever_qa_chain.invoke,
    data=dataset_name,
    evaluators=[
        cot_qa_evaluator,
        unlabeled_dopeness_evaluator,
        labeled_score_evaluator,
        unlabeled_coherence_evaluator,
        labeled_relevance_evaluator
        ],
    experiment_prefix="Parent Document Retrieval RAG Evaluation"
)

View the evaluation results for experiment: 'Parent Document Retrieval RAG Evaluation-39526c40' at:
https://smith.langchain.com/o/3e2d3fa4-8d65-5328-ab1f-a089d874c8f0/datasets/9eff5d3e-6c80-4567-811b-bd50d532ff2d/compare?selectedSessions=dac6a3f5-a605-4343-8659-d1c4cb63057c




0it [00:00, ?it/s]

Error running evaluator <DynamicRunEvaluator evaluate> on run 73362e6f-8192-40cc-b7a1-9a22890cb1b3: RateLimitError("Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-AdhcHEVRXYxcJVZh94lEyz9i on tokens per min (TPM): Limit 10000, Used 9914, Requested 382. Please try again in 1.776s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}")
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/evaluation/_runner.py", line 1233, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/evaluation/evaluator.py", line 278, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/run_helpers.py", line 568, in wrapper
    raise e
  File "/opt/

### Task 2: Ensemble Retrieval

Next let's look at ensemble retrieval!

You can read more about this [here](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)!

The basic idea is as follows:

1. Obtain User Question
2. Hit the Retriever Pair
    - Retrieve Documents with BM25 Sparse Vector Retrieval
    - Retrieve Documents with Dense Vector Retrieval Method
3. Collect and "fuse" the retrieved docs based on their weighting using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm into a single ranked list.
4. Use those documents to augment our generation.

Ensure your `weights` list - the relative weighting of each retriever - sums to 1!

In [99]:
!pip install -q -U rank_bm25

In [100]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=450, chunk_overlap=75)
split_documents = text_splitter.split_documents(documents)

bm25_retriever = BM25Retriever.from_documents(split_documents)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
vectorstore = Qdrant.from_documents(split_documents, embedding, location=":memory:")
qdrant_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

ensemble_retriever = EnsembleRetriever(
    retrievers=[
        bm25_retriever,
        qdrant_retriever
    ],
    weights=[
        0.5,
        0.5
    ])

In [101]:
ensemble_retriever_qa_chain = create_qa_chain(ensemble_retriever)

In [102]:
ensemble_retriever_qa_chain.invoke({"question" : "What is RAG?"})["response"].content

'RAG stands for Retrieval Augmented Generation, which is a central paradigm in LLM (Large Language Models) application development. It involves connecting LLMs to external data sources to address the lack of recent or private information in the models.'

In [104]:
pdr_rag_results = evaluate(
    ensemble_retriever_qa_chain.invoke,
    data=dataset_name,
    evaluators=[
        cot_qa_evaluator,
        unlabeled_dopeness_evaluator,
        labeled_score_evaluator,
        unlabeled_coherence_evaluator,
        labeled_relevance_evaluator
        ],
    experiment_prefix="Hybrid Retrieval RAG Evaluation"
)

View the evaluation results for experiment: 'Hybrid Retrieval RAG Evaluation-2b534fa8' at:
https://smith.langchain.com/o/3e2d3fa4-8d65-5328-ab1f-a089d874c8f0/datasets/9eff5d3e-6c80-4567-811b-bd50d532ff2d/compare?selectedSessions=d2122664-d804-41f2-9f93-1a4aed9b919e




0it [00:00, ?it/s]

Error running evaluator <DynamicRunEvaluator evaluate> on run 9a7d4b46-b6e1-4899-9da6-bd4b79ef8e34: RateLimitError("Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-AdhcHEVRXYxcJVZh94lEyz9i on tokens per min (TPM): Limit 10000, Used 9849, Requested 350. Please try again in 1.194s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}")
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/evaluation/_runner.py", line 1233, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/evaluation/evaluator.py", line 278, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/langsmith/run_helpers.py", line 568, in wrapper
    raise e
  File "/opt/

#### ❓Question #1:

What conclusions can you draw about the above results?

Describe in your own words what the metrics are expressing.